## This notebook shows the full workflow for building models, simulating growth and obtaining SCFA predictions from data collected by the _ex vivo_ study conducted by Gurry et al. 2021

In [64]:
import pandas as pd
import numpy as np 
import micom as mm
from plotnine import *
import os
import sys

pd.options.mode.chained_assignment = None  # default='warn'


%matplotlib inline

## First, we pull in the taxonomy table, matching each feature ID in the qiime2 output to a microbial taxa at the species level. We will build our models at the genus level, so collapse to this rank

In [ ]:
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/data/qiime2/taxonomy/data')
taxa = pd.read_csv('taxonomy.tsv',sep='\t') # read table
taxa.set_index('Feature ID',inplace=True)
taxa = taxa.Taxon.str.split(';',expand=True) # split ranks
taxa = taxa.rename(columns = {0:'Kingdom',1:'Phylum',2:'Class',3:'Order',4:'Family',5:'Genus',6:'Species'})
taxa = taxa.dropna(subset = ['Genus']) # drop undefined columns
taxa = taxa.drop(columns = ['Species']) # drop species column
taxa = taxa.apply(lambda column: column.str.split('_').str[2]) # remove prefixes
taxa = (taxa.apply(lambda row: ";".join(row.str.capitalize().fillna("")), axis=1)
        .to_frame().rename(columns = {0:'taxon'})) # join columns into taxon identifier
taxa

## Next we'll pull in the abundance table, with read counts for all present taxa. We'll drop those that aren't identified in the taxon list, and sum together duplicates. 

In [ ]:
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/data/qiime2/')
unrarefied_table = q2.Artifact.load('table.qza') # read table
abundance = unrarefied_table.view(pd.DataFrame).reset_index().rename(columns = {'index':'sample_id'})
to_drop = (abundance[abundance.columns[1:]].columns[~abundance[abundance.columns[1:]].
                                                    columns.isin(taxa.index)].to_list()) # taxa to drop
abundance = abundance.drop(columns = to_drop)
abundance = abundance.rename(columns = taxa['taxon'].to_dict())
abundance = abundance.groupby(axis=1, level=0).sum() 
abundance = pd.melt(abundance, id_vars = 'sample_id', value_vars = abundance.columns[:-1],
                    var_name = 'id', value_name = 'abundance') # melt into long form df 
abundance['genus'] = abundance['id'].str.split(';').str[-1] # need a genus column in df 
abundance

## We also need a model database to pull our reconstructions from 

In [ ]:
agora = ('/proj/gibbons/refs/micom_dbs/agora103_genus.qza')
agora

## Now, we'll build our models, with cutoff of 0.001

In [ ]:
models = mm.workflows.build(abundance,out_folder = '/proj/gibbons/nbohmann/exvivo/gurry1/micom/16S/16S_models/',
                      model_db = agora, cutoff = 0.001, threads = 20)

## We can take a look at the resulting model manifest

In [65]:
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/micom/16S/')
manifest = pd.read_csv('16S_models/manifest.csv')
manifest

,sample_id,file,found_taxa,total_taxa,found_fraction,found_abundance_fraction
0,H008,H008.pickle,30.0,50.0,0.600000,0.791525
1,H009,H009.pickle,33.0,44.0,0.750000,0.908978
2,H010,H010.pickle,38.0,56.0,0.678571,0.799867
3,H012,H012.pickle,28.0,43.0,0.651163,0.868381
4,H019,H019.pickle,22.0,33.0,0.666667,0.812679
5,H020,H020.pickle,26.0,34.0,0.764706,0.839140
6,H021,H021.pickle,28.0,38.0,0.736842,0.840889
7,H025,H025.pickle,27.0,43.0,0.627907,0.894737
8,H028,H028.pickle,20.0,25.0,0.800000,0.945193
9,H029,H029.pickle,37.0,58.0,0.637931,0.851163


In [66]:
min_med = mm.workflows.media.minimal_media(manifest,model_folder = '16S_models', summarize = True, min_growth = 0.1)

Output()

[12:52:24] WARNING  solver encountered an error infeasible                    ]8;id=151788;file:///users/nbohmann/miniconda3/envs/micom2022/lib/python3.8/site-packages/micom/solution.py\solution.py]8;;\:]8;id=431360;file:///users/nbohmann/miniconda3/envs/micom2022/lib/python3.8/site-packages/micom/solution.py#220\220]8;;\

           WARNING  minimization of medium was unsuccessful                      ]8;id=495143;file:///users/nbohmann/miniconda3/envs/micom2022/lib/python3.8/site-packages/micom/media.py\media.py]8;;\:]8;id=119359;file:///users/nbohmann/miniconda3/envs/micom2022/lib/python3.8/site-packages/micom/media.py#236\236]8;;\

           ERROR    Could not get a minimal medium for sample H029.               ]8;id=512130;file:///users/nbohmann/miniconda3/envs/micom2022/lib/python3.8/site-packages/micom/workflows/media.py\media.py]8;;\:]8;id=459068;file:///users/nbohmann/miniconda3/envs/micom2022/lib/python3.8/site-packages/micom/workflows/media.py#36\36]8;;\

OptimizationError: Could not find a growth medium that allows the specified growth rate for all taxa in all samples :(

In [ ]:
min_med

## Load in the minimal media from the previous ex vivos

In [ ]:
os.chdir('/proj/gibbons/nbohmann/exvivo/')
medium = pd.read_csv('minimal_media.csv',index_col =0)
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/micom/16S/')
manifest = pd.read_csv('16S_models/manifest.csv')
medium = mm.workflows.fix_medium(manifest,'16S_models',medium,community_growth=0.1,min_growth = 0.001,
                                     minimize_components=False, weights='C',summarize=True)
pectin_medium = medium.append({'reaction':'EX_pect_m','flux':300.00},ignore_index=True)
inulin_medium = (medium[~medium.reaction.str.contains('inulin')].
                 append({'reaction':'EX_inulin_m','flux':10.00},ignore_index=True))
fos_medium = medium.append({'reaction':'EX_kesto_m','flux':100.00},ignore_index=True)

In [ ]:
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/micom/16S/')
tradeoff = mm.workflows.tradeoff(manifest, model_folder='16S_models',medium = fos_medium, 
                                tradeoffs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
                                 presolve = True, threads = 10)

In [ ]:
from micom.viz import plot_tradeoff

In [ ]:
os.chdir('/users/nbohmann/ex_vivo/gurry1')
plot_tradeoff(tradeoff, filename='tradeoff_fos.html')

In [ ]:
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/micom/16S/')
ctrl_growth = mm.workflows.grow(manifest, model_folder='16S_models',medium = medium, 
                                tradeoff = 0.6, strategy = 'none', threads = 5)
pectin_growth = mm.workflows.grow(manifest, model_folder='16S_models',medium = pectin_medium, 
                                   tradeoff = 0.6, strategy = 'none', threads = 5)
inulin_growth = mm.workflows.grow(manifest, model_folder='16S_models/',medium = inulin_medium, 
                                   tradeoff = 0.6, strategy = 'none', threads = 5)
fos_growth = mm.workflows.grow(manifest, model_folder='16S_models',medium = fos_medium, 
                                   tradeoff = 0.6, strategy = 'none', threads = 5)

In [ ]:
def get_fluxes(growth, cond):
    growth = growth.exchanges
    growth = growth[growth.direction == "export"].groupby(["sample_id", "metabolite", "reaction"]).apply(lambda df: sum(df.flux * df.abundance)).reset_index()
    growth = growth[(growth.reaction.str.startswith('EX_but(e)'))|(growth.reaction.str.startswith('EX_ppa(e)'))]
    growth['index'] = growth['sample_id']+'_'+cond
    return growth

In [ ]:
predicted = pd.DataFrame()
predicted = pd.concat(predicted, get_fluxes(ctrl_growth, 'CTRL'))
predicted = pd.concat(predicted, get_fluxes(inulin_growth, 'INUL'))
predicted = pd.concat(predicted, get_fluxes(pectin_growth, 'PECT'))
predicted = pd.concat(predicted, get_fluxes(fos_growth, 'FOS'))
predicted = pd.pivot_table(predicted, index = 'index',columns = 'reaction', values = 0)
but = predicted['EX_but(e)'].to_dict()
ppa = predicted['EX_ppa(e)'].to_dict()

In [ ]:
ctrl = ctrl_growth.exchanges
ctrl = ctrl[ctrl.direction == "export"].groupby(["sample_id", "metabolite", "reaction"]).apply(lambda df: sum(df.flux * df.abundance)).reset_index()
ctrl = ctrl[(ctrl.reaction.str.startswith('EX_but(e)'))|(ctrl.reaction.str.startswith('EX_ppa(e)'))]
ctrl['index'] = ctrl['sample_id']+'_CTRL'
pectin = pectin_growth.exchanges
pectin = pectin[pectin.direction == "export"].groupby(["sample_id", "metabolite", "reaction"]).apply(lambda df: sum(df.flux * df.abundance)).reset_index()
pectin = pectin[(pectin.reaction.str.startswith('EX_but(e)'))|(pectin.reaction.str.startswith('EX_ppa(e)'))]
pectin['index'] = pectin['sample_id']+'_PECT'
inulin = inulin_growth.exchanges
inulin = inulin[inulin.direction == "export"].groupby(["sample_id", "metabolite", "reaction"]).apply(lambda df: sum(df.flux * df.abundance)).reset_index()
inulin = inulin[(inulin.reaction.str.startswith('EX_but(e)'))|(inulin.reaction.str.startswith('EX_ppa(e)'))]
inulin['index'] = inulin['sample_id']+'_INUL'
fos = fos_growth.exchanges
fos = fos[fos.direction == "export"].groupby(["sample_id", "metabolite", "reaction"]).apply(lambda df: sum(df.flux * df.abundance)).reset_index()
fos = fos[(fos.reaction.str.startswith('EX_but(e)'))|(fos.reaction.str.startswith('EX_ppa(e)'))]
fos['index'] = fos['sample_id']+'_FOS'
predicted = ctrl.append(pectin,ignore_index = True)
predicted = predicted.append(inulin,ignore_index = True)
predicted = predicted.append(fos,ignore_index = True)
predicted = pd.pivot_table(predicted, index = 'index',columns = 'reaction', values = 0)
but = predicted['EX_but(e)'].to_dict()
ppa = predicted['EX_ppa(e)'].to_dict()

In [ ]:
def flux_calculate(arg):
    os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/data/gc_data/')
    file = pd.read_csv(arg,index_col = 0)
    file = file[['but','ppa']].dropna()
    file = file[~file.index.str.contains("QC")]
    file['sample'] = file.index.str.split('-').str[0]
    file['treatment'] = file.index.str.split('-').str[1]
    file['timepoint'] = file.index.str.split('-').str[2]
    file['replicate'] = file.index.str.split('-').str[4]
    file = file.dropna()
    baseline = file[file.timepoint.str.contains('0')]
    baseline['treatment'] = 'INUL'
    file = file.append(baseline)
    baseline['treatment'] = 'PECT'
    file = file.append(baseline)
    baseline['treatment'] = 'FOS'
    file = file.append(baseline)
    file = file[(file.index.str.contains('CTRL'))|(file.index.str.contains('PECT'))
                |(file.index.str.contains('INUL'))|(file.index.str.contains('FOS'))]
    file = file[~file.timepoint.str.contains('2')]
    file = file.sort_values(by=['sample','treatment','replicate','timepoint'])
    file.set_index(['sample','treatment','replicate','timepoint'],inplace = True)
    file = file.groupby(['sample','treatment','replicate']).diff().dropna().reset_index()
    file = file.groupby(['sample','treatment']).mean().reset_index()
    return file

In [ ]:
sample_list = ['H008-a.csv','H009-a.csv','H010-a.csv','H012-a.csv','H019-a.csv',
               'H020-a.csv','H021-a.csv','H025-a.csv','H028-a.csv','H029-a.csv']
os.chdir('/proj/gibbons/nbohmann/exvivo/gurry1/data/gc_data/')
flux = pd.DataFrame([])
for x in sample_list:
    flux = flux.append(flux_calculate(x))
flux.reset_index(inplace = True,drop = True)
flux['index'] = flux['sample']+'_'+flux['treatment']
flux.set_index('index',inplace = True)
flux['predicted_but'] = flux.index.map(but)
flux['predicted_ppa'] = flux.index.map(ppa)

In [ ]:
plt1=(
    ggplot(
        flux,aes(x='ppa',y='predicted_ppa'))
        +geom_point(aes(color='treatment'),size=5)
        +geom_smooth(aes(group = 'treatment',color = 'treatment'),method='lm',linetype='--')
#         +geom_text(aes(label = 'sample'),nudge_y = 1)
        +labs(title='Propionate',x='Measured Propionate Production ($\dfrac{mmol}{L*h}$)',
              y = 'Predicted Propionate ($\dfrac{mmol}{gDCW*h}$)')
        +theme(text = element_text(size=15),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),
                                panel_grid=element_line(size = .2, linetype = "solid",colour = "gray"),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                legend_title=element_blank(),
                                legend_position='right'))
plt1

In [ ]:
plt1=(
    ggplot(
        flux,aes(x='but',y='predicted_but'))
        +geom_point(aes(color='treatment'),size=5)
        +geom_smooth(aes(group = 'treatment',color = 'treatment'),method='lm',linetype='--')
#         +geom_text(aes(label = 'sample'),nudge_y = 1)
        +ylim(0,80)
        +labs(title='Butyrate',x='Measured Butyrate Flux ($\dfrac{mmol}{L*h}$)',
             y = 'Predicted Butyrate ($\dfrac{mmol}{gDCW*h}$)')
        +theme(text = element_text(size=15),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),
                                panel_grid=element_line(size = .2, linetype = "solid",colour = "gray"),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                legend_title=element_blank(),
                                legend_position='right'))
plt1